In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('auto-mpg.csv')
df.head()

In [ ]:
df.info()

### Verificar Valores Nulos

In [ ]:
# Verificar valores nulos
df.isnull().sum()

In [ ]:
# Contar valores '?' en horsepower
print("\nNúmero de valores '?' en horsepower:", (df['horsepower'] == '?').sum())

### Remmplazar '?' con NaN y Convertir a Numerico

In [ ]:
import numpy as np
# Reemplazar '?' por NaN y convertir a numérico
df['horsepower'] = df['horsepower'].replace('?', np.nan)
df['horsepower'] = pd.to_numeric(df['horsepower'])

In [ ]:
# Verificar valores nulos después de la conversión
df.isnull().sum()

In [ ]:
# Estadísticas descriptivas de horsepower
print("\nEstadísticas de horsepower antes de la imputación:")
df['horsepower'].describe()

In [ ]:
from sklearn.impute import SimpleImputer

# Imputar valores nulos en horsepower con la mediana
imputer = SimpleImputer(strategy='median')
df['horsepower'] = imputer.fit_transform(df[['horsepower']])

# Verificar valores nulos después de la imputación
df.isnull().sum()

Técnica elegida: SimpleImputer con mediana, porque:

La mediana (93.5) es robusta frente a valores atípicos (máximo 230, sesgo moderado).
Solo 6 valores faltantes (1.5%) justifican una imputación simple.
La mediana es adecuada para la distribución sesgada de horsepower. Se resolvieron los 6 valores nulos originales.

In [ ]:
# Generar valores nulos en weight (5% de las filas)
np.random.seed(42)  # Para reproducibilidad
mask = np.random.choice([True, False], size=df.shape[0], p=[0.05, 0.95])
df.loc[mask, 'weight'] = np.nan

# Verificar valores nulos después de generarlos
df.isnull().sum()

In [ ]:
# Estadísticas descriptivas de weight
print("\nEstadísticas de weight antes de la imputación:")
df['weight'].describe()

In [ ]:
# Imputar valores nulos en weight
imputer = SimpleImputer(strategy='median')
df['weight'] = imputer.fit_transform(df[['weight']])

# Verificar valores nulos
print("\nValores nulos después de imputar weight:")
df.isnull().sum()

In [ ]:
# Tipos de datos
df.dtypes

## 2. Conversion de Atributos con OneHotEncoder

In [ ]:
# Número de valores únicos en car name
print("\nNúmero de valores únicos en car name:", df['car name'].nunique())

# Extraer la marca
df['brand'] = df['car name'].apply(lambda x: x.split()[0])

# Verificar marcas
print("\nNúmero de marcas únicas:", df['brand'].nunique())
print("\nFrecuencia de las 5 marcas más comunes:")
df['brand'].value_counts().head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Codificar brand
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
brand_encoded = encoder.fit_transform(df[['brand']])
brand_encoded_df = pd.DataFrame(brand_encoded, columns=encoder.get_feature_names_out(['brand']))

# Concatenar y eliminar columnas
df = pd.concat([df, brand_encoded_df], axis=1)
df = df.drop(['brand', 'car name'], axis=1)

# Mostrar primeras filas
print("\nPrimeras filas después de codificar brand:")
df.head()

In [ ]:
# Valores únicos en origin
print("\nValores únicos en origin:", df['origin'].unique())

## 3. Esacalado de Atributos con StandardScaler

In [ ]:
num_cols = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']
print("\nEstadísticas descriptivas de las columnas numéricas:")
df[num_cols].describe()

print("\nSesgo (skewness):")
df[num_cols].skew()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Escalar columnas numéricas (excepto mpg y brand_*)
num_cols_to_scale = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']
scaler = StandardScaler()
df[num_cols_to_scale] = scaler.fit_transform(df[num_cols_to_scale])

# Mostrar estadísticas
print("\nEstadísticas después de StandardScaler:")
df[num_cols_to_scale].describe()

 ## 4. Seleccion de Atributos

In [ ]:
# Separar X e y
X = df.drop('mpg', axis=1)
y = df['mpg']
print("\nDimensiones de X:", X.shape)  # (398, 41)

In [ ]:
# Columnas numéricas + mpg
num_cols = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']
corr_matrix = df[num_cols].corr()

# Visualizar matriz de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Matriz de Correlación')
plt.show()

# Correlaciones con mpg
print("\nCorrelaciones con mpg:")
print(corr_matrix['mpg'].sort_values(ascending=False))

In [ ]:
# Modelo base
model = LinearRegression()

# RFE para 6 atributos
rfe = RFE(estimator=model, n_features_to_select=6)
rfe.fit(X, y)

# Ranking y selección
rfe_ranking = pd.DataFrame({'Feature': X.columns, 'Ranking': rfe.ranking_, 'Selected': rfe.support_})
print("\nRanking de características (RFE, 6 atributos):")
print(rfe_ranking.sort_values(by='Ranking').head(10))

# Características seleccionadas
selected_features_rfe = X.columns[rfe.support_].tolist()
print("\nAtributos seleccionados por RFE:", selected_features_rfe)

In [ ]:
# Árbol de decisión
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X, y)

# Importancia de características
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': tree.feature_importances_})
print("\nImportancia de características (Árbol de Decisión):")
print(feature_importance.sort_values(by='Importance', ascending=False).head(10))

# Top 6
top_features_tree = feature_importance.nlargest(6, 'Importance')['Feature'].tolist()
print("\nAtributos seleccionados por Árbol de Decisión:", top_features_tree)

In [ ]:
# Seleccionar atributos
selected_features = ['weight', 'horsepower', 'model year', 'acceleration', 'brand_vw']
X_selected = X[selected_features]
print("\nDimensiones de X_selected:", X_selected.shape)  # (398, 5)

# Modelado

In [ ]:
# Asumimos X_selected y y ya definidos
print("Dimensiones de X_selected:", X_selected.shape)  # (398, 5)
print("Primeras filas de X_selected:\n", X_selected.head())
print("Primeras filas de y:\n", y.head())

In [ ]:
# Configurar red neuronal
nn_model = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', 
                        max_iter=1000, random_state=42, early_stopping=True, validation_fraction=0.1)

# K-Fold Cross-Validation (k=5)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# R²
nn_r2_scores = cross_val_score(nn_model, X_selected, y, cv=kf, scoring='r2')
nn_r2_mean = nn_r2_scores.mean()
nn_r2_std = nn_r2_scores.std()
nn_r2_var = nn_r2_std ** 2

# RMSE
nn_rmse_scores = np.sqrt(-cross_val_score(nn_model, X_selected, y, cv=kf, scoring='neg_mean_squared_error'))
nn_rmse_mean = nn_rmse_scores.mean()
nn_rmse_std = nn_rmse_scores.std()
nn_rmse_var = nn_rmse_std ** 2

print("\nRed Neuronal (MLPRegressor) - Resultados de Validación Cruzada (k=5):")
print(f"R² - Media: {nn_r2_mean:.4f}, Desviación Estándar: {nn_r2_std:.4f}, Varianza: {nn_r2_var:.4f}")
print(f"RMSE - Media: {nn_rmse_mean:.4f}, Desviación Estándar: {nn_rmse_std:.4f}, Varianza: {nn_rmse_var:.4f}")

In [ ]:
# Configurar Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# R²
rf_r2_scores = cross_val_score(rf_model, X_selected, y, cv=kf, scoring='r2')
rf_r2_mean = rf_r2_scores.mean()
rf_r2_std = rf_r2_scores.std()
rf_r2_var = rf_r2_std ** 2

# RMSE
rf_rmse_scores = np.sqrt(-cross_val_score(rf_model, X_selected, y, cv=kf, scoring='neg_mean_squared_error'))
rf_rmse_mean = rf_rmse_scores.mean()
rf_rmse_std = rf_rmse_scores.std()
rf_rmse_var = rf_rmse_std ** 2

print("\nRandom Forest Regressor - Resultados de Validación Cruzada (k=5):")
print(f"R² - Media: {rf_r2_mean:.4f}, Desviación Estándar: {rf_r2_std:.4f}, Varianza: {rf_r2_var:.4f}")
print(f"RMSE - Media: {rf_rmse_mean:.4f}, Desviación Estándar: {rf_rmse_std:.4f}, Varianza: {rf_rmse_var:.4f}")

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [ ]:
# Entrenar Red Neuronal
nn_model.fit(X_train, y_train)
y_pred_nn = nn_model.predict(X_test)

# Entrenar Random Forest
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Métricas en conjunto de prueba
nn_r2_test = r2_score(y_test, y_pred_nn)
nn_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_nn)) 
rf_r2_test = r2_score(y_test, y_pred_rf)
rf_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_rf))  

print("\nMétricas en conjunto de prueba:")
print(f"Red Neuronal - R²: {nn_r2_test:.4f}, RMSE: {nn_rmse_test:.4f}")
print(f"Random Forest - R²: {rf_r2_test:.4f}, RMSE: {rf_rmse_test:.4f}")

In [ ]:
# === Porcentaje dentro de ±2 mpg (Conjunto de Prueba) ===
nn_within_2mpg = np.mean(np.abs(y_test - y_pred_nn) <= 2.0) * 100
rf_within_2mpg = np.mean(np.abs(y_test - y_pred_rf) <= 2.0) * 100

In [ ]:
print("\n=== Red Neuronal ===")
print("R² promedio (5-fold):", nn_r2_mean * 100)
print("R² por fold:", nn_r2_scores)
print("Desviación estándar de R²:", round(nn_r2_std, 4))
print("Promedio de R²:", round(nn_r2_mean, 4))
print("Varianza de R²:", round(nn_r2_var, 4))
print("RMSE promedio (5-fold):", nn_rmse_mean)
print("RMSE por fold:", nn_rmse_scores)
print("Desviación estándar de RMSE:", round(nn_rmse_std, 4))
print("Promedio de RMSE:", round(nn_rmse_mean, 4))
print("Varianza de RMSE:", round(nn_rmse_var, 4))
print("Conjunto de Prueba - R²:", 0.8799)
print("Conjunto de Prueba - RMSE:", 2.5416)
print("Conjunto de Prueba - % predicciones dentro de ±2 mpg:", nn_within_2mpg)

In [ ]:
print("\n=== Random Forest ===")
print("R² promedio (5-fold):", rf_r2_mean * 100)
print("R² por fold:", rf_r2_scores)
print("Desviación estándar de R²:", round(rf_r2_std, 4))
print("Promedio de R²:", round(rf_r2_mean, 4))
print("Varianza de R²:", round(rf_r2_var, 4))
print("RMSE promedio (5-fold):", rf_rmse_mean)
print("RMSE por fold:", rf_rmse_scores)
print("Desviación estándar de RMSE:", round(rf_rmse_std, 4))
print("Promedio de RMSE:", round(rf_rmse_mean, 4))
print("Varianza de RMSE:", round(rf_rmse_var, 4))
print("Conjunto de Prueba - R²:", 0.8967)
print("Conjunto de Prueba - RMSE:", 2.3569)
print("Conjunto de Prueba - % predicciones dentro de ±2 mpg:", rf_within_2mpg)

## Gráfica de Dispersión

In [ ]:
import matplotlib.pyplot as plt

# Gráfica de dispersión
plt.figure(figsize=(12, 5))

# Red Neuronal
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_nn, color='blue', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valores Reales (mpg)')
plt.ylabel('Predicciones (mpg)')
plt.title('Red Neuronal: Dispersión de Predicciones')
plt.grid(True)

# Random Forest
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_rf, color='green', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valores Reales (mpg)')
plt.ylabel('Predicciones (mpg)')
plt.title('Random Forest: Dispersión de Predicciones')
plt.grid(True)

plt.tight_layout()
plt.show()

## Gráfica de Residuos

In [ ]:
import matplotlib.pyplot as plt

# Residuos
nn_residuals = y_test - y_pred_nn
rf_residuals = y_test - y_pred_rf

plt.figure(figsize=(12, 5))

# Red Neuronal
plt.subplot(1, 2, 1)
plt.scatter(y_pred_nn, nn_residuals, color='blue', alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicciones (mpg)')
plt.ylabel('Residuos')
plt.title('Red Neuronal: Gráfica de Residuos')
plt.grid(True)

# Random Forest
plt.subplot(1, 2, 2)
plt.scatter(y_pred_rf, rf_residuals, color='green', alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicciones (mpg)')
plt.ylabel('Residuos')
plt.title('Random Forest: Gráfica de Residuos')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
joblib.dump(rf_model, 'modelo_insectos.pkl')